In [276]:
from glob import glob
from pathlib import Path
import ROOT

In [277]:
absolute_path_only = '/media/droc/Seagate/PHYS-461/4-hall_effect/data/'
input_files = glob(absolute_path_only + '/*.root')

namer = []
for i in range(len(input_files)):

    base = Path(input_files[i]).stem
    namer.append(base[:base.index("_run")])


In [279]:
line = ROOT.TF1('line', '[0]*x+[1]', 1, 11)

In [295]:

pzero = []
pzero_error = []
pone = []
pone_error = []
title = []

def linear_fit(file_path,titlee):

    file = ROOT.TFile(file_path) 

    tree = file.Get("data")
    rdf = ROOT.RDataFrame(tree)

    bmeas = rdf.AsNumpy(columns=["Bmeas"])["Bmeas"]
    bnom = rdf.AsNumpy(columns=["Bnom"])["Bnom"][:-1]
    icoil = rdf.AsNumpy(columns=["Icoil"])["Icoil"]
    vhall = rdf.AsNumpy(columns=["VHall"])["VHall"][:-1]
    stddev = rdf.AsNumpy(columns=["stddev"])["stddev"][:-1]

    # print(bmeas)
    # print(bnom)
    # print(stddev)
    # print(vhall)
    # print(icoil)
    file.Close()

    c0 = ROOT.TCanvas("c0","", 800,800,800,600)

    c0.Update()

    latex = ROOT.TLatex()

    gr = ROOT.TGraphErrors(len(bnom),bnom,vhall,0,stddev)
    gr.SetMarkerStyle(ROOT.kFullCircle)
    gr.SetTitle(titlee)
    gr.Fit(line)

    p0 = line.GetParameter(0)
    p0_error = line.GetParError(0)
    p1 = line.GetParameter(1)
    p1_error = line.GetParError(1)

    pzero.append(line.GetParameter(0))
    pzero_error.append(line.GetParError(0))
    pone.append(line.GetParameter(1))
    pone_error.append(line.GetParError(1))
    title.append(titlee)

    gr.Draw("AP")
    latex.DrawLatexNDC(0.5,0.8,f'#scale[0.6]{{#bf{{slope: {p0:.2e} #sigma = #pm {p0_error:.2e} }}}}') 
    latex.DrawLatexNDC(0.5,0.76,f'#scale[0.6]{{#bf{{y-intercept: {p1:.2e} #sigma = #pm {p1_error:.2e}}}}}') 
    c0.Draw()

    png_file = file_path.replace(".root",".pdf")
    c0.SaveAs(png_file)
    c0.Close()

    return pzero,pzero_error,pone,pone_error,title

In [298]:
for index, file in enumerate(input_files):
    p0,p0_error,p1,p1_error,title = linear_fit(file,namer[index])

print(len(p0))
print(len(title))

62
62
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       1.7592
NDf                       =            5
Edm                       =  2.96788e-16
NCalls                    =           36
p0                        = -0.000261863   +/-   8.56797e-05 
p1                        =    0.0484334   +/-   0.000472953 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.82626
NDf                       =            7
Edm                       =   1.1677e-19
NCalls                    =           34
p0                        = -0.000335545   +/-   7.1717e-05  
p1                        =    0.0612413   +/-   0.000462729 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.71827
NDf                       =            3
Edm                       =  1.71672e-19
NCalls                    =           34
p0                        = -0.000718632 

Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Cu_6A_run1.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Cu_7.5A_run2.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Cu_9A_run3.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Fe_6A_run1.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Fe_7.5A_run2.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Fe_9A_run3.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Mo_3A_run1.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effect/data/Mo_6A_run1.pdf has been created
Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/4-hall_effec